In [26]:
import numpy as np
import argparse
import os
import glob

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

import tensorflow as tf

from azureml.core import Run
from utils import load_data, one_hot_encode
from sklearn.metrics import precision_score

In [1]:
from azureml.core.run import Run
from azureml.core import Dataset, Datastore, Workspace
import os
import argparse
import joblib
import json
from keras_mnist import preprocess_df, train_model, get_model_metrics

In [34]:
from dataclasses import dataclass
import os
from typing import Optional

from dotenv import load_dotenv


@dataclass(frozen=True)
class Env:
    """Loads all environment variables into a predefined set of properties
    """

    # to load .env file into environment variables for local execution
    load_dotenv()
    workspace_name: Optional[str] = os.environ.get("WORKSPACE_NAME")
    resource_group: Optional[str] = os.environ.get("RESOURCE_GROUP")
    subscription_id: Optional[str] = os.environ.get("SUBSCRIPTION_ID")
    tenant_id: Optional[str] = os.environ.get("TENANT_ID")
    app_id: Optional[str] = os.environ.get("SP_APP_ID")
    app_secret: Optional[str] = os.environ.get("SP_APP_SECRET")
    vm_size: Optional[str] = os.environ.get("AML_COMPUTE_CLUSTER_CPU_SKU")
    compute_name: Optional[str] = os.environ.get("AML_COMPUTE_CLUSTER_NAME")
    vm_priority: Optional[str] = os.environ.get(
        "AML_CLUSTER_PRIORITY", "lowpriority"
    )  # NOQA: E501
    min_nodes: int = int(os.environ.get("AML_CLUSTER_MIN_NODES", 0))
    max_nodes: int = int(os.environ.get("AML_CLUSTER_MAX_NODES", 4))
    build_id: Optional[str] = os.environ.get("BUILD_BUILDID")
    pipeline_name: Optional[str] = os.environ.get("TRAINING_PIPELINE_NAME")
    sources_directory_train: Optional[str] = os.environ.get(
        "SOURCES_DIR_TRAIN"
    )  # NOQA: E501
    train_script_path: Optional[str] = os.environ.get("TRAIN_SCRIPT_PATH")
    evaluate_script_path: Optional[str] = os.environ.get(
        "EVALUATE_SCRIPT_PATH"
    )  # NOQA: E501
    register_script_path: Optional[str] = os.environ.get(
        "REGISTER_SCRIPT_PATH"
    )  # NOQA: E501
    model_name: Optional[str] = os.environ.get("MODEL_NAME")
    experiment_name: Optional[str] = os.environ.get("EXPERIMENT_NAME")
    model_version: Optional[str] = os.environ.get("MODEL_VERSION")
    image_name: Optional[str] = os.environ.get("IMAGE_NAME")
    db_cluster_id: Optional[str] = os.environ.get("DB_CLUSTER_ID")
    score_script: Optional[str] = os.environ.get("SCORE_SCRIPT")
    build_uri: Optional[str] = os.environ.get("BUILD_URI")
    dataset_name: Optional[str] = os.environ.get("DATASET_NAME")
    datastore_name: Optional[str] = os.environ.get("DATASTORE_NAME")
    dataset_version: Optional[str] = os.environ.get("DATASET_VERSION")
    run_evaluation: Optional[str] = os.environ.get("RUN_EVALUATION", "true")
    allow_run_cancel: Optional[str] = os.environ.get(
        "ALLOW_RUN_CANCEL", "true"
    )  # NOQA: E501
    aml_env_name: Optional[str] = os.environ.get("AML_ENV_NAME")
    aml_env_train_conda_dep_file: Optional[str] = os.environ.get(
        "AML_ENV_TRAIN_CONDA_DEP_FILE", "conda_dependencies.yml"
    )
    rebuild_env: Optional[bool] = os.environ.get(
        "AML_REBUILD_ENVIRONMENT", "false"
    ).lower().strip() == "true"

    use_gpu_for_scoring: Optional[bool] = os.environ.get(
        "USE_GPU_FOR_SCORING", "false"
    ).lower().strip() == "true"
    aml_env_score_conda_dep_file: Optional[str] = os.environ.get(
        "AML_ENV_SCORE_CONDA_DEP_FILE", "conda_dependencies_scoring.yml"
    )
    aml_env_scorecopy_conda_dep_file: Optional[str] = os.environ.get(
        "AML_ENV_SCORECOPY_CONDA_DEP_FILE", "conda_dependencies_scorecopy.yml"
    )
    vm_size_scoring: Optional[str] = os.environ.get(
        "AML_COMPUTE_CLUSTER_CPU_SKU_SCORING"
    )
    compute_name_scoring: Optional[str] = os.environ.get(
        "AML_COMPUTE_CLUSTER_NAME_SCORING"
    )
    vm_priority_scoring: Optional[str] = os.environ.get(
        "AML_CLUSTER_PRIORITY_SCORING", "lowpriority"
    )
    min_nodes_scoring: int = int(
        os.environ.get("AML_CLUSTER_MIN_NODES_SCORING", 0)
    )  # NOQA: E501
    max_nodes_scoring: int = int(
        os.environ.get("AML_CLUSTER_MAX_NODES_SCORING", 4)
    )  # NOQA: E501
    rebuild_env_scoring: Optional[bool] = os.environ.get(
        "AML_REBUILD_ENVIRONMENT_SCORING", "false"
    ).lower().strip() == "true"
    scoring_datastore_storage_name: Optional[str] = os.environ.get(
        "SCORING_DATASTORE_STORAGE_NAME"
    )
    scoring_datastore_access_key: Optional[str] = os.environ.get(
        "SCORING_DATASTORE_ACCESS_KEY"
    )
    scoring_datastore_input_container: Optional[str] = os.environ.get(
        "SCORING_DATASTORE_INPUT_CONTAINER"
    )
    scoring_datastore_input_filename: Optional[str] = os.environ.get(
        "SCORING_DATASTORE_INPUT_FILENAME"
    )
    scoring_datastore_output_container: Optional[str] = os.environ.get(
        "SCORING_DATASTORE_OUTPUT_CONTAINER"
    )
    scoring_datastore_output_filename: Optional[str] = os.environ.get(
        "SCORING_DATASTORE_OUTPUT_FILENAME"
    )
    scoring_dataset_name: Optional[str] = os.environ.get(
        "SCORING_DATASET_NAME"
    )  # NOQA: E501
    scoring_pipeline_name: Optional[str] = os.environ.get(
        "SCORING_PIPELINE_NAME"
    )  # NOQA: E501
    aml_env_name_scoring: Optional[str] = os.environ.get(
        "AML_ENV_NAME_SCORING"
    )  # NOQA: E501
    aml_env_name_score_copy: Optional[str] = os.environ.get(
        "AML_ENV_NAME_SCORE_COPY"
    )  # NOQA: E501
    batchscore_script_path: Optional[str] = os.environ.get(
        "BATCHSCORE_SCRIPT_PATH"
    )  # NOQA: E501
    batchscore_copy_script_path: Optional[str] = os.environ.get(
        "BATCHSCORE_COPY_SCRIPT_PATH"
    )  # NOQA: E501

In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: mlops-aml-ws
Azure region: centralus
Subscription id: 67c8a1c5-3a57-45c1-afa7-2e780c1372da
Resource group: mlops-rg


In [13]:
web_paths = ['https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
             'https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [21]:
dataset = dataset.register(workspace = ws,
                            name = 'mnist-dataset',
                            description='training and test dataset',
                            create_new_version=True)

In [23]:
Dataset.get_by_name(ws,"mnist-dataset")

{
  "source": [
    "https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz",
    "https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz"
  ],
  "definition": [
    "GetFiles"
  ],
  "registration": {
    "id": "98c44951-5d10-489a-a7d4-3162d322cfb9",
    "name": "mnist-dataset",
    "version": 1,
    "description": "training and test dataset",
    "workspace": "Workspace.create(name='mlops-aml-ws', subscription_id='67c8a1c5-3a57-45c1-afa7-2e780c1372da', resource_group='mlops-rg')"
  }
}

In [26]:
xxx = os.environ.get("WORKSPACE_NAME")

In [29]:
from dotenv import load_dotenv

In [30]:
load_dotenv()

True

In [35]:
os.environ

environ{'LANG': 'C.UTF-8',
        'PATH': '/anaconda/envs/azureml_py36/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin',
        'HOME': '/home/azureuser',
        'LOGNAME': 'azureuser',
        'USER': 'azureuser',
        'SHELL': '/bin/bash',
        'INVOCATION_ID': '2d04616353fe4390aaea4c047c79dd36',
        'JOURNAL_STREAM': '9:33375',
        'MKL_THREADING_LAYER': 'GNU',
        'AML_CloudName': 'AzureCloud',
        'CI_RESOURCE_GROUP': 'mlops-rg',
        'CI_WORKSPACE': 'mlops-aml-ws',
        'CI_NAME': 'temp',
        'CI_LOCAL_UBUNTU_USER': 'azureuser',
        'DEFAULT_IDENTITY_CLIENT_ID': 'clientid',
        'APPSETTING_WEBSITE_SITE_NAME': 'AMLComputeInstance',
        'MSI_ENDPOINT': 'http://127.0.1.1:46808/MSI/auth',
        'MSI_SECRET': 'M5n3fxed9x6lVuO9hMPY',
        'JPY_PARENT_PID': '2911',
        'TERM': 'xterm-color',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        '

In [36]:
e = Env()

In [38]:
print(e.workspace_name)

None


In [12]:
model_name = args.model_name
step_output_path = args.step_output
dataset_version = args.dataset_version
data_file_path = args.data_file_path
dataset_name = args.dataset_name

In [15]:
# Load the training parameters from the parameters file
with open("parameters.json") as f:
    pars = json.load(f)
try:
    train_args = pars["training"]
except KeyError:
    print("Could not load training values from file")
    train_args = {}

In [16]:
train_args

{'n_inputs': 784,
 'n_h1': 300,
 'n_h2': 100,
 'n_outputs': 10,
 'n_epochs': 20,
 'batch_size': 50,
 'learning_rate': 0.001}

In [18]:
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]
train_args["n_inputs"]

784

In [39]:
data_folder = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/temp/code/Users/lydia.ingabire.1/mnist/data'

In [43]:
glob.glob(os.path.join(data_folder, '**/train-images-idx3-ubyte.gz'), recursive=True)[0]

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/temp/code/Users/lydia.ingabire.1/mnist/data/train-images-idx3-ubyte.gz'

In [44]:
datastore = Datastore.get(aml_workspace, datastore_name)
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, file_path))
dataset = dataset.register(workspace=aml_workspace,
                               name=dataset_name,
                               create_new_version=True)

NameError: name 'Datastore' is not defined